#                                IMFAT
######           Interface-Managed Financial Analysis Tool


For this project we have developed a financial tool that interacts with two live API's (CoinGecko and FinnHub) to extract various data on stocks, crypto and fiat and provide visualization of the financial metrics made available through the API's. The tool utilizes a dashboard for selecting the securities and financial plots required to perform the analysis. Additionaly, a conversion tool has been developed enabling conversion between fiat/fiat, crypto/crypto, crypto/fiat and fiat/crypto which is also performed via user-interface. And for a complete interactive app, a tool has been integrated to the application allowing it to send text messages directly to the cellphone of Twilio account-holders when a monitored security hits a set value .

In [1]:
# Import Libraries, Modules & Dependencies
import panel as pn
import requests

In [2]:
# Initialize Panel Resources
pn.extension()

# Get CoinGecko coins list
def get_coingecko_coins():
    url = 'https://api.coingecko.com/api/v3/coins/list'
    response = requests.get(url)
    data = response.json()
    coin_dict = {coin['symbol'].upper(): coin['id'] for coin in data}
    return coin_dict

coin_dict = get_coingecko_coins()

In [3]:
# Define a list of fiat currency codes
currencies = ['USD', 'EUR', 'JPY', 'GBP', 'AUD', 'CAD', 'CHF', 'CNY', 'SEK', 'NZD', 'SGD']

# Get price from CoinGecko - conditions to identify and store accordingly the inputs of from and to currency as they are in the currencies list or the coin_dict
def get_price(from_currency, to_currency):
    if from_currency in coin_dict:
        from_currency_id = coin_dict[from_currency]
    else:
        from_currency_id = from_currency.lower()

    if to_currency in coin_dict:
        to_currency_id = coin_dict[to_currency]
    else:
        to_currency_id = to_currency.lower()
    
# conditions for converting curr/curr must include a conv to usd first the api link does not accept not having usd on either side of the link variables

    if from_currency in currencies and to_currency in currencies:
        vs_currency = from_currency_id
        usd_currency = 'usd'
        url = f'https://api.coingecko.com/api/v3/simple/price?ids={usd_currency}&vs_currencies={vs_currency}'
        response = requests.get(url)
        data = response.json()
        from_currency_rate = data[usd_currency][vs_currency]

        vs_currency = to_currency_id
        url = f'https://api.coingecko.com/api/v3/simple/price?ids={usd_currency}&vs_currencies={vs_currency}'
        response = requests.get(url)
        data = response.json()
        to_currency_rate = data[usd_currency][vs_currency]
        
        conversion_rate = to_currency_rate / from_currency_rate
    
# conditions for converting from/to tkn/curr

    elif from_currency in currencies:
        vs_currency = from_currency.lower()
        url = f'https://api.coingecko.com/api/v3/simple/price?ids={to_currency_id}&vs_currencies={vs_currency}' #Only takes in seq of tkn then curncy
        response = requests.get(url)
        data = response.json()
        conversion_rate = 1 / data[to_currency_id][vs_currency] # Swapped with below

    elif to_currency in currencies:
        vs_currency = to_currency.lower()
        url = f'https://api.coingecko.com/api/v3/simple/price?ids={from_currency_id}&vs_currencies={vs_currency}'
        response = requests.get(url)
        data = response.json()
        conversion_rate = data[from_currency_id][vs_currency] # Swapped with above-WN
    
# condition for tkn/tkn conversion, get both values in usd then calc conversion rate    
    else:
        vs_currency = 'usd'
        url = f'https://api.coingecko.com/api/v3/simple/price?ids={from_currency_id},{to_currency_id}&vs_currencies={vs_currency}'
        response = requests.get(url)
        data = response.json()
        from_currency_rate = data[from_currency_id][vs_currency]
        to_currency_rate = data[to_currency_id][vs_currency]
        conversion_rate = from_currency_rate / to_currency_rate # Swapped

    return conversion_rate
    #return vs_currency
    #return to_currency_id

In [6]:
get_price('AUD','JPY')

90.9054054054054

In [5]:
# Create widgets
from_currency_widget = pn.widgets.AutocompleteInput(name='From: Fiat or Crypto', options=list(coin_dict.keys()) + currencies, value='AUD')
to_currency_widget = pn.widgets.AutocompleteInput(name='To: Crypto or Fiat', options=list(coin_dict.keys()) + currencies, value='XRP')
amount_widget = pn.widgets.FloatInput(name='Amount', value=1.0)

# Modify the convert_currencies function to use CoinGecko API for both crypto and fiat conversions
def convert_currencies(from_currency, to_currency, amount):
    if from_currency == to_currency:
        return amount
    else:
        price = get_price(from_currency, to_currency)
        return round(amount * price, 8)

# Update the conversion result when the button is clicked
def update_conversion_result(event):
    from_currency = from_currency_widget.value
    to_currency = to_currency_widget.value
    amount = amount_widget.value
    result = convert_currencies(from_currency, to_currency, amount)
    conversion_result_widget.value = f'{amount} {from_currency.upper()} = {result} {to_currency.upper()}' #Swapped


# Create result widget and button with cell-like appearance
conversion_result_widget = pn.widgets.TextInput(name='Result', value='', disabled=True)
convert_button = pn.widgets.Button(name='Convert', button_type='primary', background='#ADD8E6')
convert_button.on_click(update_conversion_result)

# Create the currency converter panel
currency_converter = pn.Column("Dynamic Currency Converter", from_currency_widget, to_currency_widget, amount_widget, convert_button, conversion_result_widget)

# Display the dashboard in a pop-up window
dashboard = pn.Tabs(('Converter', currency_converter))
dashboard.show()

Launching server at http://localhost:61484
